# Boost Switching regulator LM3478
- Supply: 20 V
- Output voltage: 40 V (theoretically up to 45 V)
	- Make the output voltage variable through some variable resistor (Manually / voltage controlled)

In [4]:

import math
#############! DESIGN PARAMETERS ##################
### Take from LTSpice simulation + add on top
I_peak_A = 3 # Amps, for each bridge spike of 1 amp 180 degrees shifted
I_rms_A = math.sqrt(I_peak_A) # Amps (slightly overdimensioned already)
print(f"I_rms_A: {I_rms_A}")

### DIODE Choice:
# 40 V max
# RMS voltage max: 28 V
# 5 A RMS current max
V_diode = 0.85 # SS510 (Max current: 5 Amps, Max voltage: 100 V)
V_max_diode = 100 # 

### MOSFET Choice:
# NCE6005AS-mosfet -> usable in full bridge as well as sepic + MUCH cheaper
Vfet_gsth = 1.6 # Volt
Vfet_diode = 1.2 # Volt
Rfet_dsn = 32e-3 # At 3 amps, 2.5 V
Vfet_dsn = Rfet_dsn * I_rms_A
Vds_max = 60 # V
Vgs_max = 20 # V
Id_max = 5 # A
Id_peak = 24 # A (pulsed drain current)
print(f"Vfet_dsn {Vfet_dsn}")
freq_sw = 500e3

# Input voltage
Vin = 5

# Required Output Voltage
Vout = 10

I_rms_A: 1.7320508075688772
Vfet_dsn 0.05542562584220407


In [5]:

###! >>> Chose Inductor
L_ch = 50e-6

###! <<< Chose Inductor

D_boost = 1 - (Vin-Rfet_dsn*I_rms_A) / (Vout+V_diode)
print(f"Duty cycle: {D_boost}")

print(f"\r\n#### Inductor Selection ####\r\n")
L_min = D_boost * (1-D_boost) * Vin / (2 * I_rms_A * freq_sw)
print(f"L_min: {L_min*1e6} uH")

dIl_max = D_boost * Vin / (2 * freq_sw * L_ch)
Il_avg = I_rms_A / (1-D_boost)
Il_peak = Il_avg + dIl_max
print(f"Il_peak: {Il_peak}, Il_avg: {Il_avg}, dIl: {dIl_max}")
V_sense = 156e-3 # Volt
Vsl = 92e-3
ISW_limit = (I_rms_A / (1-D_boost) + (D_boost * Vin) / (2 * freq_sw * L_ch))
print(f"ISW_limit: {ISW_limit}")
print(f"\r\n#### Rsense selection ####\r\n")
R_sense = (V_sense - D_boost * Vsl) / ISW_limit
R_sense_max = 2 * Vsl * freq_sw * L_ch / (Vout - 2 * (Vin-0.01))
print(f"R_sense: {R_sense} should be < {R_sense_max}")
Rsl = (R_sense * (Vout - 2 * Vin) / (2*freq_sw * L_ch) - Vsl) / (40e-3)
if (Rsl < 0):
	print(f"No RSL addition necessary!")
else:
	print(f"RSL should be > {Rsl}")


print(f"\r\n#### DIODE selection ####\r\n")
Idiode_peak = I_rms_A / (1-D_boost) + dIl_max
print(f"Diode peak current: {Idiode_peak}")
print(f"\r\n#### INPUT CAPACITOR SELECTION ####\r\n")
Icin_rms = (Vout - Vin) / (2 * math.sqrt(3) * Vout * L_ch * freq_sw) * Vin
print(f"Cin RMS current: {Icin_rms} A")

print(f"\r\n#### OUTPUT CAPACITOR SELECTION ####\r\n")
Icout_rms = math.sqrt((1-D_boost) * (I_rms_A**2 * D_boost / (1-D_boost)**2 + dIl_max**2 / 3))
print(f"Icout_rms {Icout_rms} A")

Duty cycle: 0.5442788595246271

#### Inductor Selection ####

L_min: 0.7160280215663788 uH
Il_peak: 3.8551091660926295, Il_avg: 3.800681280140167, dIl: 0.0544278859524627
ISW_limit: 3.8551091660926295

#### Rsense selection ####

R_sense: 0.027476872991147133 should be < 230.00000000000492
No RSL addition necessary!

#### DIODE selection ####

Diode peak current: 3.8551091660926295

#### INPUT CAPACITOR SELECTION ####

Cin RMS current: 0.02886751345948129 A

#### OUTPUT CAPACITOR SELECTION ####

Icout_rms 1.8929931561470625 A


### Inductor high current (3-5 amps)
- Since the boost converter takes about 5 amps through its inductor, we need either a more expensive inductor, or 2 boost-stages which can perform current sharing naturally through CCM.
- However CCM doesn't mean that the current is shared perfectly.
- Try finding a smaller set of inductors with a higher current rating if possible, that might be usable for all cases (also for the SEPIC's, since they will need it to get from 5 -> 10 V at 1 A rms)